# Introduction
This [Jupyter](https://jupyter.org/) notebook can be run using [colab.research.google.com](https://colab.research.google.com) (see [here](https://colab.research.google.com/notebooks/intro.ipynb) for an intro) or from [jupyter.org/try](https://jupyter.org/try) (select JupyterLab). Alternatively this source file can be downloaded and run locally with [Anaconda](https://docs.anaconda.com/anaconda/navigator/) or directly with Jupyter (Lab|Notebook). Jupyter and Anaconda should be installed in all AUT engineering and computer science labs. I recommend using a web-interface for portability.

The benefit of using Jupyter is that code snippets can be run live (Python is running in the background).

The version on Github is static; markdown is rendered but code cannot be executed. All code can be copied and pasted into your favourite text editor or IDE and *should* run with Python 3.x ;)

You are encouraged to use any programming language you feel comfortable with, this is simply an example using Python (and Jupyter is designed for Python demonstrations).

---

# Tutorial: Create a block from scratch using Python

Blocks can contain anything digital. This 'data' is what is written to the blockchain when a new block is added. This data could be prices of stocks, reddit posts, digital signatures, images (not a good idea), or---getting creative---it could even be data from another blockchain.

To be useful the data will contain many fields such as:<br>
 - identifier
 - timestamp
 - previous hash (to make the chain)
 - merkle root
 - list of transactions (the data of interest)
 
These can be stored in a python [dictionary](https://docs.python.org/3/tutorial/datastructures.html?#dictionaries) which is a key-value structure. Think of the key as an index.

```
dict = {key_1:value_1,
        key_2:value_2,
        .
        .
        .
        key_n:value_n   
}
```

## Initialize a new block. This one will be the _genesis_ block
Press ```shift+enter``` to run the individual code cell. Or select the code click the play (run) button from the toolbar. See the ```Kernal``` and ```Run``` menu for all options.

You may have to wait for the environment to initialize if this is the first time. There is a status bar above.

The output will appear directly below the code block.

In [119]:
# initialize a block. Note 'transactions' is initialized as an empty list
block = {
    'height': 1,
    'time': 0,
    'prevHash': 'this is the genesis block',
    'merkleRoot': '',
    'transactions': [],
        }
print(block)

{'height': 1, 'time': 0, 'prevHash': 'this is the genesis block', 'merkleRoot': '', 'transactions': []}


## Create a transaction and hash it

Let's create a transaction to store in our blockchain. Remember a transaction is just data; this can be anything represented as a digital object. 

In [120]:
# create a transaction, in this case a string
transaction='Pay $1,000,000 to Jeff'
print(transaction)

Pay $1,000,000 to Jeff


To store the transaction object, we will hash it to create a unique identifier of the information. First we will need access to python's hashing library. Then we will hash the 'transaction' object we created above. In the third line we will output the new hashed value.

The [hash library](https://docs.python.org/3/library/hashlib.html?#module-hashlib) has access to many standard hash functions. ```SHA-256``` or secure hashing algorithm with a 256 bit output is particularly famous in cryptocurrency.


In [121]:
# the hash library has many built-in hash functions such as SHA-1 and MD5
import hashlib as hash
hashed_tx = hash.sha256(transaction)
print(hashed_tx)


TypeError: Strings must be encoded before hashing

```p
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-5-8b54d0eb476f> in <module>()
      1 # the hash library has many built-in hash functions such as SHA-1 and MD5
      2 import hashlib as hash
----> 3 hashed_tx = hash.sha1(transaction)
      4 print(hashed_tx)

TypeError: Strings must be encoded before hashing
```

Here we have an **error in line three**, note the green arrow ```----> 3```

The error message is telling us that we cannot hash a string object such as 'Pay $1,000,000 to Jeff'. (Why not?)

First it must be encoded.

In [122]:
encoded_tx = transaction.encode()
print(encoded_tx)

b'Pay $1,000,000 to Jeff'


Note the output begins with a ```b'``` which is telling us that the string is now a byte object. We can successfully hash a byte object.

In [123]:
hashed_tx = hash.sha256(encoded_tx)
print(hashed_tx)

<sha256 _hashlib.HASH object @ 0x0000019EDD143350>


This shows a ```SHA-256``` hash object at the specified memory address. Unfortunately this isn't human-readable and we can't copy and paste it for verification elsewhere. Note that your address is different from your neighbours is different from mine: ```0x7fed...```

The `digest()` and `hexdigest()` methods will output byte objects and hex strings respectively. 

In [124]:
print(hashed_tx.digest())
print(hashed_tx.hexdigest())

b'\x17]\x8e\x86\rg\x8c\xfdX\xceo\xb9\xcd\xcfVV5e^\x05fR\xd5L\xf4\xf1#\xd2\x8aM%2'
175d8e860d678cfd58ce6fb9cdcf565635655e056652d54cf4f123d28a4d2532


Which type of object would you rather work with?

## Add the transaction to the block

We now have a hashed object. This should be unique -- only the specific string ```Pay $1,000,000 to Jeff``` should have the hashed hex output of ```175d8e860d678cfd58ce6fb9cdcf565635655e056652d54cf4f123d28a4d2532```. There also should be no **collisions** meaning that if you create a transaction and hash it you will not get the same output. The last thing to note is that this object is a **fixed length**. So if the data was very long (this whole notebook text file, or a Tolstoy novel) I would still get a 160 bit output (40 hex digits).

Lets store this transaction and add it to the ```block``` we created above.

In [125]:
hex_tx = hashed_tx.hexdigest()
block["transactions"].append(hex_tx)
print(block)

{'height': 1, 'time': 0, 'prevHash': 'this is the genesis block', 'merkleRoot': '', 'transactions': ['175d8e860d678cfd58ce6fb9cdcf565635655e056652d54cf4f123d28a4d2532']}


```transactions``` is an array ```[]``` and we can see the tx output

In [107]:
# Function to hash data
def hash_data(data):
    return hash.sha256(data.encode()).hexdigest()

## Add the current timestamp to a block

In [126]:
import time
timestamp = int(time.time())
block['time'] = timestamp
print(block)


{'height': 1, 'time': 1721948430, 'prevHash': 'this is the genesis block', 'merkleRoot': '', 'transactions': ['175d8e860d678cfd58ce6fb9cdcf565635655e056652d54cf4f123d28a4d2532']}


## Add Merkle root hash to a block

Note there was only 1 transaction and so this became the ```merkleRoot```. A proper [Merkle root](https://en.wikipedia.org/wiki/Merkle_tree) represents the root of a pairwise transaction tree where every non-leaf node holds the hash of the two child nodes. Merkle trees are very handy and a fundamental part of blockchains.

In [127]:
# Function to create a Merkle root from a list of transactions
def create_merkle_root(trans):
    if len(transactions) == 0:
        return ''
    if len(transactions) % 2 != 0:
        # If the number of transactions is odd, duplicate the last one
        transactions.append(transactions[-1])
    
    # Hash the individual transactions to form the leaf nodes
    leaf_nodes = [hash_data(t) for t in transactions]

    # Combine pairs of leaf node hashes and hash them again to form the next layer
    while len(leaf_nodes) > 1:
        next_layer = []
        for i in range(0, len(leaf_nodes), 2):
            combined_hash = hash_data(leaf_nodes[i] + leaf_nodes[i + 1])
            next_layer.append(combined_hash)
        leaf_nodes = next_layer

    # The last remaining element is the Merkle root
    return leaf_nodes[0]

In [128]:
merkle_root = create_merkle_root(transaction)
print(merkle_root)
block["merkleRoot"] = merkle_root
print(block)

b5867694af47ea08e25c71808594a2fe1d64485a840a50b8fa078c09a377a01a
{'height': 1, 'time': 1721948430, 'prevHash': 'this is the genesis block', 'merkleRoot': 'b5867694af47ea08e25c71808594a2fe1d64485a840a50b8fa078c09a377a01a', 'transactions': ['175d8e860d678cfd58ce6fb9cdcf565635655e056652d54cf4f123d28a4d2532']}


## Create a new block and append it to the chain

This block only has a single transaction (perhaps its the block reward to Jeff ;) Now we will create a new block and append it to the chain. The block is created in the same manner, except we must make a few updates:

1.   the blockheight is now incremented by 1
2.   the `prevHash` field gets the hash of the genesis block. This will ensure the state of the blockchain is preserved moving forward.

In [87]:
# some attributes have been hard-coded for simplicity
block2 = {
    'height': 2,
    'time': 0,
    'prevHash': '',
    'merkleRoot': '',
    'transactions': [],
        }
# create a transaction and add it to the block
tx = hash.sha1('Alice +10'.encode()).hexdigest()
block2["transactions"].append(tx)
block2["merkleRoot"] = tx
print(block2)

{'height': 2, 'time': 1, 'prevHash': '', 'merkleRoot': '9726fd28f4baeeef320445819ce41b02ca756e19', 'transactions': ['9726fd28f4baeeef320445819ce41b02ca756e19']}


If there are more than one transaction in the list, such that,

In [88]:
transactions = [
    "Pay $1,000,000 to Mike",
    "Pay $500,000 to Nate",
    "Pay $200,000 to Charlie",
    "Pay $300,000 to Edna"
]

In [89]:
merkle_root = create_merkle_root(transactions)
print(merkle_root)

# add transactions to the block
for j in transactions:
    block2["transactions"].append(hash_data(j))
block2["merkleRoot"] = merkle_root
print(block2)

b5867694af47ea08e25c71808594a2fe1d64485a840a50b8fa078c09a377a01a
{'height': 2, 'time': 1, 'prevHash': '', 'merkleRoot': 'b5867694af47ea08e25c71808594a2fe1d64485a840a50b8fa078c09a377a01a', 'transactions': ['9726fd28f4baeeef320445819ce41b02ca756e19', '4aab7c52959ebf64d9b44b120883237d630991bd2f3377bcab36a3626517d0a0', '342ee31a3f3476e53f128f8de3bfb7dece4c6f5849f8cd5fcdddee24ce8eaaba', 'fda8e90c916c412bdfd4a1e8a2ea4e8b19ac890af288c64073f719f79c607d89', '94a384692fd1a10e67bf50c2647dd2e2df602acc7cfd05169f35de77ffe599e8']}


The only thing left is to link the blocks. For this we need to hash the entire genesis block object. Proceeding as before:

In [94]:
hash_block_1 = hash.sha256(block.encode())

AttributeError: 'dict' object has no attribute 'encode'

Another **error**! This is a uniquely python error. We need to convert the block (dictionary) into a byte object. To do this we need to use the `pickle` functionality that is built in. You may know this as serialization. Once pickled, we can hash and store as a hex digest.

In [95]:
import pickle
# convert to a byte object
byte_genesis = pickle.dumps(block)
print(byte_genesis)

# compress to a human-readable SHA-1 digest
hash_genesis = hash.sha256(byte_genesis).hexdigest()
print('\n')
print(hash_genesis)

b'\x80\x04\x95\xd0\x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\x06height\x94K\x01\x8c\x04time\x94Ju\xe2\xa0f\x8c\x08prevHash\x94\x8c\x19this is the genesis block\x94\x8c\nmerkleRoot\x94\x8c@b5867694af47ea08e25c71808594a2fe1d64485a840a50b8fa078c09a377a01a\x94\x8c\x0ctransactions\x94]\x94\x8c(ec07e8ede0e48a7bc29e41a9f1dfe874739183db\x94au.'


3a27b28c18e7cdebb1c3dea45fd967d36504e44669c412742edea9a8511f67cc


Earlier we said hashing has the benefit of being fixed length. Here you can see the ```byte_genesis``` output is much longer than our previous byte outputs while `hash_genesis` remains short. 

Set the ```prevHash``` pointer in ```block2``` to the hash of the genesis block.

In [129]:
# set the prevHash and print the block
block2["prevHash"] = hash_genesis
block2["time"] = int(time.time())
for key, value in block2.items():
    print(key+': '+str(value)) 

height: 2
time: 1721949009
prevHash: 85ad1310fc8a476c999a7dad503f69e4a7840c7e
merkleRoot: b5867694af47ea08e25c71808594a2fe1d64485a840a50b8fa078c09a377a01a
transactions: ['9726fd28f4baeeef320445819ce41b02ca756e19', '4aab7c52959ebf64d9b44b120883237d630991bd2f3377bcab36a3626517d0a0', '342ee31a3f3476e53f128f8de3bfb7dece4c6f5849f8cd5fcdddee24ce8eaaba', 'fda8e90c916c412bdfd4a1e8a2ea4e8b19ac890af288c64073f719f79c607d89', '94a384692fd1a10e67bf50c2647dd2e2df602acc7cfd05169f35de77ffe599e8']


### That's the main concept of creating a blockchain! (datastructure)

As noted above, the consensus mechanism is a whole other part, but this (hopefully) shows that coding a blockchain is not that intense.

# Modify a transaction to attack the chain

A hash produces randomized output without any discernable pattern relating to the original data. Lets test this by modifying the transaction in the genesis block, rehashing, and comparing to the `prevHash` pointer in block2.

Changing a single transaction modifies the Merkle root which modifies the block hash and will invalidate the entire chain from that point in history forward.


In [115]:
# change the dollar sign to a negative sign in the original transaction
new_transaction = 'Pay -1,000,000 to Jeff'
hashed_new_tx=hash.sha1(new_transaction.encode()).hexdigest()
# update the block with the new tx; recall 'block' is the original or genesis block; our tx was at position 0
block["transactions"][0]=hashed_new_tx

# hash the updated block
import pickle
byte_genesis_new = pickle.dumps(block)
hash_genesis = hash.sha1(byte_genesis_new).hexdigest()

# compare hashes
if block2["prevHash"] != hash_genesis:
    print('Your chain has been attacked!!')

Your chain has been attacked!!


## Summary

In this tutorial we have:<br>
 - created a block structure including a list of transactions (data)
 - hashed the transaction and added it to the block
 - hashed the entire block
 - added a new block 
 - linked the two blocks with a previous hash field to create a block chain
 
What we have __not__ done is:<br>
 - used a real timestamp
 - use a merkle tree to store the transactions
 - store the merkle root in our block structure
 
Python libraries that this code depends on:
 - [hashlib](https://docs.python.org/3/library/hashlib.html)
 - [pickle](https://docs.python.org/3/library/pickle.html)

Other resources:
- [python 3 docs](https://docs.python.org/3/)
- [Google colab faq](https://research.google.com/colaboratory/faq.html)
- [Jupyter docs](https://docs.jupyter.org/en/latest/)

## Exercises
1. Create your blocks using a real time stamp. Is there a difference between this and an indexed method like block height?
2. Automate your block production to run continually. How do you decide the time between blocks? Comment on the factors that influence blocktime.
3. Create a merkle root of the transactions from bitcoin block [641818](https://blockchair.com/bitcoin/block/641818) mined on August 2nd, 2020. A \*.csv file of the 1870 transactions can be downloaded from this [repo](https://github.com/millecodex/COMP726/blob/master/bitcoin_block_641818.csv). In github click **raw** to see unformatted text. Just use the ```hash``` column representing the tx hash; the other data is not stored in the tree.
4. Write a script to calculate the total number of bitcoin that will be mined. Start with a block reward of 50 at height 1, and implement the halving every 210,000 blocks. The total number is just under 21 million, why not exactly 21 million? Produce a plot of supply and block production.
#### Submission
Submission is via Canvas. Prepare typed solutions, include any code with output, and plots and references that are required.